# Data pre-processing

## Setup -CHECK SPELLING MISTAKES

In [ ]:
# Install a pip package in the current Jupyter kernel - REMOVE IN PROD
#import sys
#!{sys.executable} -m pip install gtfparse

Import relevant libraries

In [2]:
# Import libs
import os.path
from os import path
import gzip
import pandas as pd
import numpy as np
import urllib.request
from gtfparse import read_gtf
from biomart import BiomartServer

Setup paths - main data path is folder inside the code repository where the data dowloaded from download-data has been stored

In [3]:
data_path = '../data'
gtf_path_gz = data_path + '/Mus_musculus.GRCm38.99.gtf.gz'
gtf_path = data_path + '/Mus_musculus.GRCm38.99.gtf'
tpm_yang_path_gz = data_path + '/GSE90848_Ana6_basal_hair_bulb_TPM.txt.gz'
tpm_yang_path = data_path + '/GSE90848_Ana6_basal_hair_bulb_TPM.txt'
gene_tsv_path = data_path + '/gene_names.tsv'

Set the main thresholds for the pre-processing
- min_num_genes_in_cell_exp - is the min number of genes that need to have expression > **min_ltpm_exp** for it to be considered a valid cell
- min_num_cells_for_gene_exp - is the min number of cells that need to have an expression for that gene > **min_ltpm_exp** for it to be considered a valid gene

In [4]:
# Thresholds
min_num_genes_in_cell_exp = 1000
min_num_cells_for_gene_exp = 10
min_ltpm_exp = 1

## Resolve Gene names

Load GTF data and show some data

In [5]:
df_gtf = read_gtf(gtf_path)
df_gtf

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'ccds_id', 'protein_id', 'protein_version']


,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,transcript_support_level,exon_number,exon_id,exon_version,ccds_id,protein_id,protein_version
0,1,havana,gene,3073253,3074322,NaN,+,0,ENSMUSG00000102693,1,...,,,,,,,,,,
1,1,havana,transcript,3073253,3074322,NaN,+,0,ENSMUSG00000102693,1,...,havana,TEC,basic,NA,,,,,,
2,1,havana,exon,3073253,3074322,NaN,+,0,ENSMUSG00000102693,1,...,havana,TEC,basic,NA,1,ENSMUSE00001343744,1,,,
3,1,ensembl,gene,3102016,3102125,NaN,+,0,ENSMUSG00000064842,1,...,,,,,,,,,,
4,1,ensembl,transcript,3102016,3102125,NaN,+,0,ENSMUSG00000064842,1,...,ensembl,snRNA,basic,NA,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868325,JH584295.1,ensembl,CDS,708,752,NaN,-,2,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,5,,,,ENSMUSP00000137004,1
1868326,JH584295.1,ensembl,exon,565,633,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,6,ENSMUSE00000997159,1,,,
1868327,JH584295.1,ensembl,CDS,565,633,NaN,-,2,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,6,,,,ENSMUSP00000137004,1
1868328,JH584295.1,ensembl,exon,66,109,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,7,ENSMUSE00001007635,1,,,


Filter GTF data for exons, 3' and 5' UTRs, call `.shape` to see how many rows we have

In [6]:
df_gtf_filt = df_gtf[(df_gtf.feature=='exon') | (df_gtf.feature=='three_prime_utr') | (df_gtf.feature=='five_prime_utr')]
df_gtf_filt.shape

(1025718, 26)

## Load in RNA-Seq data

Read the yang1 data set and look at the data

In [7]:
df_tpm_1 = pd.read_csv(tpm_yang_path, sep='\t')
df_tpm_1

,Gene_id,P1-3-A1,P1-3-A10,P1-3-A11,P1-3-A12,P1-3-A2,P1-3-A3,P1-3-A4,P1-3-A5,P1-3-A6,...,P2-1-H11,P2-1-H12,P2-1-H2,P2-1-H3,P2-1-H4,P2-1-H5,P2-1-H6,P2-1-H7,P2-1-H8,P2-1-H9
0,ENSMUSG00000000001_Gnai3,0.0,17.60,7.02,17.50,0.00,38.31,38.42,47.10,13.19,...,45.27,5.70,13.80,15.89,19.42,3.21,5.65,20.33,16.72,33.57
1,ENSMUSG00000000003_Pbsn,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ENSMUSG00000000028_Cdc45,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,8.07,0.00,0.00,0.00,2.55,0.00
3,ENSMUSG00000000031_H19,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,112.13,...,0.00,6.80,0.00,4.14,0.00,0.00,0.00,0.00,0.00,0.00
4,ENSMUSG00000000037_Scml2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.47,0.00,0.00,0.00,0.00,0.00,0.40,0.00,1.32,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46604,ERCC-00164,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
46605,ERCC-00165,0.0,0.00,42.44,3.28,0.00,0.00,9.45,9.12,0.00,...,5.41,1.76,0.00,0.00,13.15,40.04,9.40,0.00,18.05,0.00
46606,ERCC-00168,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
46607,ERCC-00170,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,2.11,0.00,0.00,0.00,0.00,0.00,0.00


Clean the yang1 dataset to get the gene names, call `.shape` to check we havent filtered anything

In [8]:
# Split gene ids on _ and load into a new data frame and set the columns
split_data = pd.DataFrame(df_tpm_1.Gene_id.str.split("_", expand=True))
split_data.columns = ["Gene_id", "Gene_name", "Gene_name2"]

# Fill in the NA's with blank strings
split_data["Gene_name"] = split_data.Gene_name.fillna('')
split_data["Gene_name2"] = split_data.Gene_name2.fillna('')

# Concatinate the strings that have split more than once back to their standard for e.g GENEID_GENENAME_SOMEMORENAME
split_data["Gene_name"] = split_data.apply(lambda x: x.Gene_name if x.Gene_name2 == '' else x.Gene_name + '_' + x.Gene_name2, axis=1)
split_data.shape

(46609, 3)

Write the gene names back into the main dataset, print out dataset to check we do indeed have gene names where they were available

In [9]:
# Insert the columns back into the main data array
df_tpm_1["Gene_id"] = split_data.Gene_id
df_tpm_1.insert(1,"Gene_name", split_data.Gene_name)
df_tpm_1

,Gene_id,Gene_name,P1-3-A1,P1-3-A10,P1-3-A11,P1-3-A12,P1-3-A2,P1-3-A3,P1-3-A4,P1-3-A5,...,P2-1-H11,P2-1-H12,P2-1-H2,P2-1-H3,P2-1-H4,P2-1-H5,P2-1-H6,P2-1-H7,P2-1-H8,P2-1-H9
0,ENSMUSG00000000001,Gnai3,0.0,17.60,7.02,17.50,0.00,38.31,38.42,47.10,...,45.27,5.70,13.80,15.89,19.42,3.21,5.65,20.33,16.72,33.57
1,ENSMUSG00000000003,Pbsn,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ENSMUSG00000000028,Cdc45,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,8.07,0.00,0.00,0.00,2.55,0.00
3,ENSMUSG00000000031,H19,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,6.80,0.00,4.14,0.00,0.00,0.00,0.00,0.00,0.00
4,ENSMUSG00000000037,Scml2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.47,0.00,0.00,0.00,0.00,0.00,0.40,0.00,1.32,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46604,ERCC-00164,,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
46605,ERCC-00165,,0.0,0.00,42.44,3.28,0.00,0.00,9.45,9.12,...,5.41,1.76,0.00,0.00,13.15,40.04,9.40,0.00,18.05,0.00
46606,ERCC-00168,,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
46607,ERCC-00170,,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,2.11,0.00,0.00,0.00,0.00,0.00,0.00


## Create merged dataset

Create merged dataset from all subsets

In [10]:
df_tpm_combined = df_tpm_1

Check gene column is unique before setting is as an index

In [11]:
df_tpm_combined.Gene_id.is_unique

True

Create data only set (this is easier for tensorflow to deal with)

In [12]:
df_tpm_combined_dataonly = df_tpm_combined
df_tpm_combined_dataonly.index = df_tpm_combined['Gene_id']
df_tpm_combined_dataonly = df_tpm_combined.drop(['Gene_id', 'Gene_name'], axis=1)

Create `log2(TPM+1)` dataset

In [13]:
df_ltpm_combined_dataonly = np.log2(df_tpm_combined_dataonly + 1)
df_ltpm_combined_dataonly

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


,P1-3-A1,P1-3-A10,P1-3-A11,P1-3-A12,P1-3-A2,P1-3-A3,P1-3-A4,P1-3-A5,P1-3-A6,P1-3-A7,...,P2-1-H11,P2-1-H12,P2-1-H2,P2-1-H3,P2-1-H4,P2-1-H5,P2-1-H6,P2-1-H7,P2-1-H8,P2-1-H9
Gene_id,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000000001,0.0,4.217231,3.003602,4.209453,0.000000,5.296824,5.300856,5.587965,3.826803,3.414136,...,5.532005,2.744161,3.887525,4.078097,4.351911,2.073820,2.733354,4.414812,4.147307,5.111449
ENSMUSG00000000003,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000000028,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3.181103,0.000000,0.000000,0.000000,1.827819,0.000000
ENSMUSG00000000031,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.821838,0.000000,...,0.000000,2.963474,0.000000,2.361768,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSMUSG00000000037,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.304511,0.000000,0.000000,0.000000,0.000000,0.000000,0.485427,0.000000,1.214125,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERCC-00164,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ERCC-00165,0.0,0.000000,5.440952,2.097611,0.000000,0.000000,3.385431,3.339137,0.000000,5.599318,...,2.680324,1.464668,0.000000,0.000000,3.822730,5.358959,3.378512,0.000000,4.251719,0.000000
ERCC-00168,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Filter for genes and cells which match filtering criteria. Call `.shape` to check what was filtered

In [14]:
df_expression_filt_mask = df_ltpm_combined_dataonly > min_ltpm_exp
df_ltpm_combined_dataonly_genefilt = df_ltpm_combined_dataonly[df_expression_filt_mask.sum(axis=1) > min_num_cells_for_gene_exp]
df_ltpm_combined_dataonly_genefilt.shape

(17139, 384)

In [15]:
df_ltpm_combined_dataonly_genecellfilt = df_ltpm_combined_dataonly_genefilt \
    .T[(df_ltpm_combined_dataonly_genefilt > min_ltpm_exp).sum(axis=0) > min_num_genes_in_cell_exp]
df_ltpm_combined_dataonly_genecellfilt = df_ltpm_combined_dataonly_genecellfilt.T
df_ltpm_combined_dataonly_genecellfilt.shape

(17139, 380)

## Write data to file

Get the column and row names as a list

In [16]:
df_column_names = pd.DataFrame(list(df_ltpm_combined_dataonly_genecellfilt.columns.values))
df_row_names = pd.DataFrame(list(df_ltpm_combined_dataonly_genecellfilt.index.values))

print(df_column_names.shape)
print(df_row_names.shape)

(380, 1)
(17139, 1)


Write the data to file

In [17]:
df_ltpm_combined_dataonly_genecellfilt.to_csv(data_path + '/tpm_combined.csv', index=False, header=False)
df_column_names.to_csv(data_path + '/tpm_combined_cols.csv', index=False, header=False)
df_row_names.to_csv(data_path + '/tpm_combined_rows.csv', index=False, header=False)